In [1]:
import pandas as pd
import os

category_attributes = pd.read_parquet('category_attributes.parquet')
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

print("CATEGORY ATTRIBUTES")
print(category_attributes.head())
print("\n")
print(category_attributes.describe())
print("\n")

print("TRAIN DATA")
print(train_data.head(),"\n")
print(train_data.describe(),"\n")


CATEGORY ATTRIBUTES
              Category  No_of_attribute  \
0          Men Tshirts                5   
1               Sarees               10   
2               Kurtis                9   
3        Women Tshirts                8   
4  Women Tops & Tunics               10   

                                      Attribute_list  
0  [color, neck, pattern, print_or_pattern_type, ...  
1  [blouse_pattern, border, border_width, color, ...  
2  [color, fit_shape, length, occasion, ornamenta...  
3  [color, fit_shape, length, pattern, print_or_p...  
4  [color, fit_shape, length, neck_collar, ocassi...  


       No_of_attribute
count         5.000000
mean          8.400000
std           2.073644
min           5.000000
25%           8.000000
50%           9.000000
75%          10.000000
max          10.000000


TRAIN DATA
   id     Category  len      attr_1 attr_2   attr_3   attr_4         attr_5  \
0   0  Men Tshirts    5     default  round  printed  default  short sleeves   
1   1  Men 

In [2]:
print(train_data.isnull().sum())

id              0
Category        0
len             0
attr_1      18346
attr_2      15021
attr_3      15515
attr_4      10325
attr_5      13720
attr_6      32097
attr_7      28798
attr_8      32739
attr_9      36648
attr_10     45214
dtype: int64


In [3]:
for column in train_data.columns[2:]:
    train_data[column] = train_data[column].fillna(train_data[column].mode()[0])

print("Missing values filled.")


Missing values filled.


In [4]:
print(train_data.isnull().sum())

id          0
Category    0
len         0
attr_1      0
attr_2      0
attr_3      0
attr_4      0
attr_5      0
attr_6      0
attr_7      0
attr_8      0
attr_9      0
attr_10     0
dtype: int64


In [5]:

category_attributes['Attribute_list'] = category_attributes['Attribute_list'].apply(
    lambda x: [attr.lower() for attr in x]
)
category_attributes.head()


,Category,No_of_attribute,Attribute_list
0,Men Tshirts,5,"[color, neck, pattern, print_or_pattern_type, ..."
1,Sarees,10,"[blouse_pattern, border, border_width, color, ..."
2,Kurtis,9,"[color, fit_shape, length, occasion, ornamenta..."
3,Women Tshirts,8,"[color, fit_shape, length, pattern, print_or_p..."
4,Women Tops & Tunics,10,"[color, fit_shape, length, neck_collar, ocassi..."


In [6]:
import os
from PIL import Image
import torchvision.transforms as transforms

# Define the transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Pre-trained model normalization
])

def preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")
    return transform(image)

train_image_folder = "train_images/"
test_image_folder = "test_images/"

preprocessed_train_images = {
    str(img_id): preprocess_image(os.path.join(train_image_folder, f"{img_id:06d}.jpg"))
    for img_id in train_data['id'][:2000]
}

preprocessed_test_images = {
    str(img_id): preprocess_image(os.path.join(test_image_folder, f"{img_id:06d}.jpg"))
    for img_id in test_data['id'][:750]
}


In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from sklearn.metrics import f1_score

# Define the model with added novelty
class AttributeClassifier(nn.Module):
    def __init__(self, num_attributes, num_categories):
        super(AttributeClassifier, self).__init__()
        self.feature_extractor = models.resnet18(pretrained=True)
        num_features = self.feature_extractor.fc.in_features
        self.feature_extractor.fc = nn.Identity()  

        embedding_dim = 50
        self.category_embedding = nn.Embedding(num_categories, embedding_dim)

        self.attribute_heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(num_features + embedding_dim, 128),
                nn.BatchNorm1d(128),
                nn.ReLU(),
                nn.Dropout(0.5),  # Dropout layer for regularization
                nn.Linear(128, 1)
            ) for _ in range(num_attributes)
        ])

    def forward(self, images, categories):
        features = self.feature_extractor(images)
        category_embeds = self.category_embedding(categories)
        combined = torch.cat([features, category_embeds], dim=1)
        
        # Use a residual connection for the outputs
        outputs = torch.cat([head(combined).squeeze(1).unsqueeze(1) for head in self.attribute_heads], dim=1)
        return torch.sigmoid(outputs)  

In [16]:
# Define parameters
num_attributes = 77  # Update this based on your dataset
num_categories = len(subset_train_data['Category'].unique())  # Extracted dynamically from dataset
model = AttributeClassifier(num_attributes, num_categories).to(device)

# Evaluation metric
from sklearn.metrics import f1_score

def compute_f1_scores(y_true, y_pred):
    y_pred_binary = (y_pred > 0.5).astype(int)
    macro_f1 = f1_score(y_true, y_pred_binary, average='macro',zero_division=1)
    micro_f1 = f1_score(y_true, y_pred_binary, average='micro',zero_division=1)
    return macro_f1, micro_f1


In [ ]:
pip install scikit-learn

In [17]:
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim

In [18]:
from sklearn.preprocessing import MultiLabelBinarizer

# Filter only attribute columns (attr_1 to attr_10)
attribute_cols = [col for col in train_data.columns if col.startswith('attr_')]

# Preprocessing function for each row
def preprocess_attributes(row):
    return [str(x).strip() for x in row if pd.notnull(x) and str(x).strip()]

# Apply row-wise to create lists of strings
formatted_labels = train_data[attribute_cols].apply(preprocess_attributes, axis=1)

# Combine all unique attributes across rows for binarization
unique_classes = sorted(set(attr for row in formatted_labels for attr in row))

# Use MultiLabelBinarizer with the unique classes
mlb = MultiLabelBinarizer(classes=unique_classes)
encoded_labels = mlb.fit_transform(formatted_labels)

# Confirm the output
encoded_df = pd.DataFrame(encoded_labels, columns=mlb.classes_)
train_data_encoded = pd.concat([train_data.reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)

print(f"Classes: {mlb.classes_}")
print(f"Encoded labels shape: {encoded_labels.shape}")


Classes: ['a-line' 'applique' 'big border' 'black' 'blue' 'botanical' 'boxy'
 'calf length' 'casual' 'checked' 'cream' 'crop' 'cuffed sleeves' 'daily'
 'default' 'elephant' 'ethnic motif' 'fitted' 'floral' 'funky print'
 'graphic' 'green' 'grey' 'high' 'jacquard' 'knee length' 'knitted' 'long'
 'long sleeves' 'loose' 'maroon' 'multicolor' 'navy blue' 'net' 'no'
 'no border' 'orange' 'party' 'peach' 'peacock' 'pink' 'polo' 'printed'
 'puff sleeves' 'purple' 'quirky' 'red' 'regular' 'regular sleeves'
 'round' 'round neck' 'ruffles' 'same as border' 'same as saree'
 'short sleeves' 'sleeveless' 'small border' 'solid' 'square neck'
 'straight' 'stylised' 'sweetheart neck' 'tassels and latkans'
 'temple border' 'three-quarter sleeves' 'tie-ups' 'traditional'
 'typography' 'v-neck' 'waist tie-ups' 'wedding' 'white' 'woven design'
 'yellow' 'yes' 'zari' 'zari woven']
Encoded labels shape: (70213, 77)


In [19]:
formatted_labels

0        [default, round, printed, default, short sleev...
1        [multicolor, polo, solid, solid, short sleeves...
2        [default, polo, solid, solid, short sleeves, s...
3        [multicolor, polo, solid, solid, short sleeves...
4        [multicolor, polo, solid, solid, short sleeves...
                               ...                        
70208    [multicolor, fitted, regular, square neck, cas...
70209    [yellow, regular, crop, round neck, casual, de...
70210    [maroon, fitted, crop, round neck, casual, sol...
70211    [default, regular, regular, high, casual, shor...
70212    [pink, boxy, crop, v-neck, casual, printed, ty...
Length: 70213, dtype: object

In [20]:
class AttributeDataset(Dataset):
    def __init__(self, dataframe, preprocessed_images, encoded_labels, category_encoder):
        """
        Args:
            dataframe: DataFrame containing metadata (e.g., ID, Category).
            preprocessed_images: Dictionary mapping image IDs to tensors.
            encoded_labels: Encoded attribute labels (e.g., numpy array or tensor).
            category_encoder: Encoder for converting categories to numerical labels.
        """
        self.dataframe = dataframe
        self.preprocessed_images = preprocessed_images
        self.encoded_labels = encoded_labels
        self.category_encoder = category_encoder

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        # Retrieve metadata row
        row = self.dataframe.iloc[idx]

        # Load the image
        img_id = str(row['id'])  # Ensure ID is string to match dictionary keys
        if img_id in self.preprocessed_images:
            image = self.preprocessed_images[img_id]
        else:
            # Log missing image and use a placeholder tensor
            print(f"Image not found for ID: {img_id}. Using dummy tensor.")
            image = torch.zeros(3, 224, 224)  # Dummy tensor for missing images

        # Encode the category
        category = row['Category']
        category = torch.tensor(self.category_encoder.transform([category])[0], dtype=torch.long)

        # Load the pre-encoded attribute labels
        labels = torch.tensor(self.encoded_labels[idx], dtype=torch.float32)

        return image, category, labels


In [10]:
subset_train_data = train_data.head(2000) 
subset_encoded_labels = encoded_labels[:2000]

In [11]:
print(len(subset_train_data))

2000


In [21]:
# Use all training data
from sklearn.preprocessing import LabelEncoder

category_encoder=LabelEncoder()
category_encoder.fit(subset_train_data['Category'])
train_dataset = AttributeDataset(subset_train_data,preprocessed_train_images, encoded_labels,category_encoder=category_encoder)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Check the size of the dataset and loader
print("Size of train dataset:", len(train_dataset))
print("Size of train loader (number of batches):", len(train_loader))

Size of train dataset: 2000
Size of train loader (number of batches): 63


In [22]:
import pandas as pd

# Load the dataset
train_df = pd.read_csv('train.csv')

# Get unique categories from the 'Category' column and create a mapping to indices
category_to_index = {category: idx for idx, category in enumerate(train_df['Category'].unique())}
print(category_to_index)  # This will show the actual category to index mapping


{'Men Tshirts': 0, 'Sarees': 1, 'Kurtis': 2, 'Women Tshirts': 3, 'Women Tops & Tunics': 4}


In [15]:
device = torch.device("cpu")


In [23]:
import torch.optim as optim
import numpy as np
# Define parameters
num_attributes = 77  # Update this based on your dataset
num_categories = len(subset_train_data['Category'].unique())  # Extracted dynamically from dataset
model = AttributeClassifier(num_attributes, num_categories).to(device)

class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        BCE_loss = nn.BCEWithLogitsLoss()(inputs, targets)
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1 - pt) ** self.gamma * BCE_loss
        return F_loss
        
criterion = FocalLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
# Training loop
epochs = 5
model.train()
for epoch in range(epochs):
    running_loss = 0.0
    all_labels = []
    all_outputs = []
    
    for batch in train_loader:
        # Unpack the batch
        images, categories, labels = batch

        # Move tensors to the correct device
        images, categories, labels = images.to(device), categories.to(device), labels.to(device)

        optimizer.zero_grad()  # Reset gradients
        outputs = model(images, categories)  # Forward pass
        loss = criterion(outputs, labels.float())  # Compute loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights

        running_loss += loss.item()
        
        all_labels.append(labels.cpu().numpy())
        all_outputs.append(outputs.cpu().detach().numpy())

    # Compute F1 scores after each epoch
    all_labels = np.concatenate(all_labels)
    all_outputs = np.concatenate(all_outputs)
    macro_f1, micro_f1 = compute_f1_scores(all_labels, all_outputs)

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(train_loader)}, Macro F1: {macro_f1}, Micro F1: {micro_f1}")



/home/kanav-arora/Desktop/AI/path/to/venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/kanav-arora/Desktop/AI/path/to/venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/5, Loss: 0.3335119594657232, Macro F1: 0.10175008783322144, Micro F1: 0.21120347269615028
Epoch 2/5, Loss: 0.3262584696686457, Macro F1: 0.11650526760244438, Micro F1: 0.24549554924468414
Epoch 3/5, Loss: 0.3196418441477276, Macro F1: 0.12414578979555049, Micro F1: 0.2752946578907087
Epoch 4/5, Loss: 0.31274342678842093, Macro F1: 0.12901821398417818, Micro F1: 0.31143156501599206
Epoch 5/5, Loss: 0.3042297533580235, Macro F1: 0.13261293400466495, Micro F1: 0.35593582887700537


In [38]:
print(f"Categories: {categories}, Type: {type(categories)}")

Categories: ('Men Tshirts', 'Men Tshirts', 'Men Tshirts', 'Men Tshirts', 'Men Tshirts', 'Men Tshirts', 'Men Tshirts', 'Men Tshirts', 'Men Tshirts', 'Men Tshirts', 'Men Tshirts', 'Men Tshirts', 'Men Tshirts', 'Men Tshirts', 'Men Tshirts', 'Men Tshirts', 'Men Tshirts', 'Men Tshirts', 'Men Tshirts', 'Men Tshirts', 'Men Tshirts', 'Men Tshirts', 'Men Tshirts', 'Men Tshirts', 'Men Tshirts', 'Men Tshirts', 'Men Tshirts', 'Men Tshirts', 'Men Tshirts', 'Men Tshirts', 'Men Tshirts', 'Men Tshirts'), Type: <class 'tuple'>


In [30]:
import torch
from torchvision import transforms
from PIL import Image
import torch.nn.functional as F

# Define the preprocessing transformation (adjust based on model requirements)
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize image to 224x224
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize based on ImageNet stats
])

# Load the image
test_image_path = '/home/kanav-arora/Desktop/AI/test_images/000008.jpg'
test_image = Image.open(test_image_path).convert('RGB')  # Ensure it's RGB format

# Preprocess the image
test_image = preprocess(test_image).unsqueeze(0).to(device)  # Add batch dimension and move to device

# Assume the category is known (replace with actual category index)
category_index = 0  # Replace with actual category index (you can map category names to indices)

# Convert the category to a tensor
category_tensor = torch.tensor([category_index], dtype=torch.long).to(device)

# Switch to evaluation mode and make prediction
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # No need to calculate gradients during inference
    output = model(test_image, category_tensor)  # Make prediction
    probability = torch.sigmoid(output)  # Apply sigmoid to logits to get probabilities
    prediction = (probability > 0.65).float()  # Binarize the output based on threshold (0.5)
print(prediction)

predicted_attributes = prediction.squeeze().tolist()  # Convert to list
attributes = [
    'a-line', 'applique', 'big border', 'black', 'blue', 'botanical', 'boxy',
    'calf length', 'casual', 'checked', 'cream', 'crop', 'cuffed sleeves', 'daily',
    'default', 'elephant', 'ethnic motif', 'fitted', 'floral', 'funky print',
    'graphic', 'green', 'grey', 'high', 'jacquard', 'knee length', 'knitted', 'long',
    'long sleeves', 'loose', 'maroon', 'multicolor', 'navy blue', 'net', 'no',
    'no border', 'orange', 'party', 'peach', 'peacock', 'pink', 'polo', 'printed',
    'puff sleeves', 'purple', 'quirky', 'red', 'regular', 'regular sleeves',
    'round', 'round neck', 'ruffles', 'same as border', 'same as saree',
    'short sleeves', 'sleeveless', 'small border', 'solid', 'square neck',
    'straight', 'stylised', 'sweetheart neck', 'tassels and latkans',
    'temple border', 'three-quarter sleeves', 'tie-ups', 'traditional',
    'typography', 'v-neck', 'waist tie-ups', 'wedding', 'white', 'woven design',
    'yellow', 'yes', 'zari', 'zari woven'
]
# Print each attribute with its prediction
for name, value in zip(attributes, predicted_attributes):
    print(f"{name}: {'Present' if value == 1 else 'Absent'}")

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.]])
a-line: Absent
applique: Absent
big border: Absent
black: Absent
blue: Absent
botanical: Absent
boxy: Absent
calf length: Absent
casual: Absent
checked: Absent
cream: Absent
crop: Absent
cuffed sleeves: Absent
daily: Absent
default: Present
elephant: Absent
ethnic motif: Absent
fitted: Absent
floral: Absent
funky print: Absent
graphic: Absent
green: Absent
grey: Absent
high: Absent
jacquard: Absent
knee length: Absent
knitted: Absent
long: Absent
long sleeves: Absent
loose: Absent
maroon: Absent
multicolor: Absent
navy blue: Absent
net: Absent
no: Absent
no border: Absent
orange: Absent
party: Present
peach: Absent
peacock: Absent
pink: 

In [40]:
# Assuming categories is a list of category strings
if isinstance(categories, tuple):
    # Convert each category string to its corresponding index
    category_indices = [category_to_index[cat] for cat in categories]
    categories = torch.tensor(category_indices, dtype=torch.long).to(device)

In [ ]:
from fastapi import FastAPI, File, UploadFile
import io

app = FastAPI()

@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
    image = Image.open(io.BytesIO(await file.read())).convert("RGB")
    image = transform(image).unsqueeze(0).cuda()  # Preprocess
    model.eval()
    with torch.no_grad():
        outputs = model(image)
    predictions = outputs.cpu().numpy()
    return {"attributes": predictions.tolist()}

# Run with: uvicorn filename:app --reload
